# Homework 5 - Explore California and Nevada with graphs
---
Here we import every library we'll need.

In [1]:
import networkx as nx
import numpy as np
from collections import deque

---
## Graph class
Here we're going to create our graph class which we're going to use for basically every task. Since we'll use a lot of nodes and every node shouldn't be connected to many edges (considering that our graph represents a map of roads) we'll use an djacency list to represent the graph. An adjacency matrix would take too much memory and most of the entries would be zero, so it would be really inefficient.

We're going to assume we'll work with undirected graphs, so we implement a class consequentially.

In [75]:
# This class is going to represent a graph, so that we can organize all the implementation and methods of
# our data structure in an organized way
class Graph:
    
    # This constructur takes as parameter the number of nodes
    def __init__(self, size):
        self._nodes = [[] for _ in range(size)]
        self._nodes_features = [{} for _ in range(size)]
        
    # This method adds features to a node in the form of a dictionary
    def add_features_to_node(self, node, features, replace = False):
        if(replace):
            self._nodes_features[node - 1] = features
        else:
            for key in features.keys():
                self._nodes_features[node - 1][key] = features[key]
                
    # This method is for removing all features from the nodes of the graph
    def reset_features(self):
        for index in range(len(self._nodes_features)):
            self._nodes_features[index] = {}
        
    # This method is for getting features of a node
    def get_feature_of_node(self, node, feature):
        moment = self._nodes_features[node - 1]
        if(feature in moment.keys()):
            return(moment[feature])
        return(None)
        
    # This method adds an edge to the graph, together with an optional dict of weights
    # The edge is given as a tuple of nodes
    def add_edge(self, edge, weights = None):
        to_add = (edge[1] - 1, {})
        if(not weights is None):
            to_add = (edge[1] - 1, weights)
        self._nodes[edge[0] - 1].append(to_add)
        
        to_add = (edge[0] - 1, {})
        if(not weights is None):
            to_add = (edge[0] - 1, weights)
        self._nodes[edge[1] - 1].append(to_add)
        
    # This method gets all neighbours of a node and given weights of the edges connecting them to the node
    def get_neighbours(self, node, features = None):
        if(features is None):
            return([single_node[0] + 1 for single_node in self._nodes[node - 1]])
        to_return = []
        for neighbour in self._nodes[node - 1]:
            to_return.append((neighbour[0] + 1, dict((k, neighbour[1][k]) for k in features if k in neighbour[1])))
        return(to_return)
    
    # This method get an edge from the graph together with the specified weights
    # If the edge doesn't exist it returns None
    def get_edge(self, node_one, node_two, features = None):
        for neighbour in self._nodes[node_one - 1]:
            if(neighbour[0] == node_two - 1):
                if(features is None):
                    return((node_one + 1, node_two + 1))
                return((node_one + 1, node_two + 1, dict((k, neighbour[1][k]) for k in features if k in neighbour[1])))
        return(None)
    
    # This method updates and edge adding more weights or replacing them (based on replace parameter)
    # It doesn't do anything if the edge doesn't exist
    def update_edge(self, node_one, node_two, weights):
        for neighbour in self._nodes[node_one - 1]:
            if(neighbour[0] == node_two - 1):
                for key in weights.keys():
                    neighbour[1][key] = weights[key]
                return
            
    # This method prints the adjacency list of the graph, it will be used just to check that everything works fine
    def print_graph(self):
        for index in range(len(self._nodes)):
            print(index + 1, end = " : ")
            print([(node[0] + 1, node[1]) for node in self._nodes[index]])

---
## Shortest path in a graph
In order to solve a lot of tasks we'll need a way to find the shortest path connecting two nodes in a graph. This is a common problem and we'll implement two different algorithms to accomplish our goal.

The first one is the breadth-first-search to find out what the minimum path connecting two nodes in a graph is, whitout taking into consideration any weight. Then we'll implement the Dijkstra's algorithm to find the shortest path based on a certain weight.

### Breadth-first-search
Here we implement our breadth-first-search algorithm. Since we want it to work for graphs belonging to our class we're going to implement this method like it belongs to our <code>Graph</code> class and then use <code>seatattr()</code> to actually add it to the class. Before jumping straight to the method we need to create a node/parent class which we'll use for saving our results.

In [4]:
# This class is just for containing data: a node and his parent node.
class Node_With_Parent:
    
    # Simple constructor
    def __init__(self, this_node, parent_node):
        self._node = this_node
        self._parent = parent_node
        
    # Get the node in the data
    def get_this_node(self):
        return(self._node)
    
    # Get the parent node (which is againa  node_with_parent object)
    def get_parent(self):
        return(self._parent)

Now we can define our breadth-first-search method.

In [76]:
def breadth_first_search(self, source, target):
    in_queue_nodes = deque()
    
    in_queue_nodes.appendleft(Node_With_Parent(source, None))
    while(True):
        if(not in_queue_nodes):
            self.reset_features()
            return(None)
        visiting_now = in_queue_nodes.pop()
        if(visiting_now.get_this_node() == target):
            break
        self.add_features_to_node(visiting_now.get_this_node(), {"Visited" : True})
        for neighbour in self.get_neighbours(visiting_now.get_this_node()):
            if(self.get_feature_of_node(neighbour, "Visited") is None):
                in_queue_nodes.appendleft(Node_With_Parent(neighbour, visiting_now))
                
    self.reset_features()
    
    to_return = []
    while(not visiting_now is None):
        to_return.append(visiting_now.get_this_node())
        visiting_now = visiting_now.get_parent()
    
    return(to_return[::-1])

setattr(Graph, 'breadth_first_search', breadth_first_search)